In [2]:
import pandas as pd
import numpy as np
import math
import glob
import os

# Milestone 1: Calculating ELO rating

In [3]:
wd = os.getcwd()
files = glob.glob(wd + '/*.xlsx')
for f in files:
    if "US Open" in f:
        files.remove(f)

#### New datasets: 4th Aug 2022.

In [5]:
wd_new = os.getcwd() 
files_new = glob.glob(wd_new + '/new_datasets/*.xlsx')
for f in files_new:
    if "_with_elo" in f:
        files_new.remove(f)

#### New functionalities

In [175]:
tournaments = np.array([])
rounds = np.array([])
surfaces = np.array([])
for file in files_new:
    df = pd.read_excel(file)
    tournaments = np.append(tournaments, df.Tournament.unique())
    rounds = np.append(rounds, df.Round.unique())
    surfaces = np.append(surfaces, df.Surface.unique())
tournaments = np.unique(tournaments)
rounds = np.unique(rounds)
surfaces = np.unique(surfaces)

In [176]:
surfaces

array(['Carpet', 'Clay', 'Grass', 'Hard', 'I.hard'], dtype=object)

In [170]:
for tour in tournaments.tolist():
    if 'Cup' in tour:
        print(tour)

ATP Cup - Brisbane, Perth, Sydney
ATP Cup - Melbourne
ATP Kremlin Cup - Moscow
Antalya Cup - Antalya
Bet-at-home Cup - Kitzbuhel
Kremlin Cup - Moscow
Laver Cup - Boston
Laver Cup - Chicago
Laver Cup - Geneva
Laver Cup - Prague
Mercedes Cup - Stuttgart
Power Horse Cup - Dusseldorf
Power Horse World Team Cup - Dusseldorf
Rogers Cup - Montreal
Rogers Cup - Toronto
Tennis Masters Cup - Shanghai
Tennis Masters Cup Shanghai - Shanghai


In [ ]:
round_points = {}
for round in rounds.tolist():
    

In [168]:
rounds

array(['1/2', '1/4', 'Bronze', 'Final', 'First', 'Fourth', 'Qualifying',
       'Robin', 'Second', 'Third', 'q-First', 'q-Second'], dtype=object)

In [139]:
masters_keys = ['Indian Wells', 'Miami', 'Monte Carlo', 'Madrid', 'Italian', 'Canadian', 'Cincinnati', 'Shanghai', 'Paris', 'Toronto', 'Rome']
atp500_keys = ['Rio', 'Rotterdam', 'Dubai', 'Mexican', 'Mexico', 'Barcelona', "Queen's", 'Queen', 'London', 'Halle', 'Hamburg', 'Washington',
               'China', 'Bejing', 'Japan', 'Tokyo', 'Vienna', 'Wien', 'Swiss', 'Basel']

In [86]:
masters_1000 = []
for tour in tournaments.tolist():
    for mast in masters_keys:
        if mast in tour:
            masters_1000.append(tour)
for master in masters_1000:
    if 'Challenger' in master:
        masters_1000.remove(master)
masters_1000.remove('French Open - Paris')

In [160]:
atp500 = []
for tour in tournaments.tolist():
    for t in atp500_keys:
        if t in tour:
            atp500.append(tour)
atp500 = list(dict.fromkeys(atp500))
removal_keys = ['Challenger', 'Los Cabos', 'Olympics', 'Gstaad', 'Wimbledon', 'Finals']
for t in atp500:
    for r in removal_keys:
        if r in t:
            atp500.remove(t)
atp500.remove('Olympics - Rio de Janeiro')
atp500.remove('Nitto ATP Finals - London')

In [187]:
tour_points = {}
for tour in tournaments.tolist():
    tour_points.update({tour: 0.7})
    if 'Australian Open - Melbourne' in tour:
        tour_points.update({tour: 1})
    if 'French' in tour:
        tour_points.update({tour: 1})
    if 'Wimbledon' in tour:
        tour_points.update({tour: 1})
    if 'U.S. Open' in tour:
        tour_points.update({tour: 1})
    if 'Finals' in tour:
        tour_points.update({tour: 0.9})
    if 'Olympics' in tour:
        tour_points.update({tour: 0.8})

    for mast in masters_1000:
        tour_points.update({mast: 0.85})
    for tr in atp500:
        tour_points.update({tr: 0.75})

### Function for winning probability of a player

In [5]:
def probability(elo_1, elo_2):
    
    return 1/(1 + math.pow(10, (elo_1 - elo_2)/400))

### Function for calculating ELO rating

In [6]:
def elo_rating(rating_player, rating_opponent, K):
    
    prob_a = probability(rating_opponent, rating_player)
    prob_b = probability(rating_player, rating_opponent)
    
    rating_player = rating_player + K * (1 - prob_a)
    rating_opponent = rating_opponent + K * (0 - prob_b)
    
    return rating_player, rating_opponent

In [ ]:
#df = pd.read_excel(files_new[0])

#for index, row in df.iterrows():
#    print(row.Tournament)

for i in tour_points:
       print(i)

In [ ]:
def k_factor():
    
    


### Function for applying ELO rating to an existing dataset

In [7]:
def apply_elo(df):

    K = 32
    for index, row in df.iterrows():
    #for i in range(df.shape[0]):
        for i in tour_points:
            if 
        elo_winner, elo_loser = elo_rating(df.at[index, 'winner_elo'], df.at[index, 'loser_elo'], 32)

        df.at[index, 'winner_elo'] = np.around(elo_winner)
        df.at[index, 'loser_elo'] = np.around(elo_loser)

        winner_name = df['Player 1'][index]
        loser_name = df['Player 2'][index]

        sub_ww = df.iloc[index:, :][df.iloc[index:, :]['Player 1']==winner_name].winner_elo
        sub_ll = df.iloc[index:, :][df.iloc[index:, :]['Player 2']==loser_name].loser_elo
        sub_lw = df.iloc[index:, :][df.iloc[index:, :]['Player 2']==winner_name].loser_elo
        sub_wl = df.iloc[index:, :][df.iloc[index:, :]['Player 1']==loser_name].winner_elo
        
        sub_ww.replace(to_replace=sub_ww.values.tolist(), value=np.around(elo_winner), inplace=True)
        sub_ll.replace(to_replace=sub_ll.values.tolist(), value=np.around(elo_loser), inplace=True)
        sub_lw.replace(to_replace=sub_lw.values.tolist(), value=np.around(elo_winner), inplace=True)
        sub_wl.replace(to_replace=sub_wl.values.tolist(), value=np.around(elo_loser), inplace=True)
        
        df.update(sub_ww)
        df.update(sub_ll)
        df.update(sub_lw)
        df.update(sub_wl)
        
    return df

### Function for extracting season final ELO rating

In [8]:
def season_final_elo(df):
    
    list1 = df['Player 1'].unique()
    list2 = df['Player 2'].unique()
    names = np.unique(np.concatenate((list1, list2))).tolist()
    
    my_list = []
    for name in names:
        sub_df = df[((df['Player 1']==name) | (df['Player 2']==name))]
        last = sub_df.iloc[-1,:]
        
        if last['Player 1'] != name:
            case = {'Name': name, 'elo': last.loser_elo}
            my_list.append(case)
        else:
            case = {'Name': name, 'elo': last.winner_elo}
            my_list.append(case)
            
    my_df = pd.DataFrame(data=my_list)
            
    return my_df, names

### Function for applying ELO rating of the earlier season to the next season

In [9]:
def apply_elo_on_new_season(df, df_year_earlier):
    
    my_df, names = season_final_elo(df_year_earlier)

    for name in names:

        sub_df_win = df[df['Player 1']==name].winner_elo
        sub_df_loss = df[df['Player 2']==name].loser_elo

        sub_df_win = sub_df_win.replace(to_replace=sub_df_win.values.tolist(), value=float(my_df[my_df['Name']==name].elo.values))
        sub_df_loss = sub_df_loss.replace(to_replace=sub_df_loss.values.tolist(), value=float(my_df[my_df['Name']==name].elo.values))

        df.update(sub_df_win)
        df.update(sub_df_loss)
        
    return df

### Function for datetime formatting, dataframe sorting and initializing ELOs 

In [27]:
def df_preprocessing(file):
    
    df = pd.read_excel(file)
    
    for i in range(df.shape[0]):
        df.at[i, 'Player 1'] = df.at[i, 'Player 1'].split(') ')[-1]
        df.at[i, 'Player 2'] = df.at[i, 'Player 2'].split(') ')[-1]
    
    #df.tourney_date = pd.to_datetime(df.tourney_date, yearfirst=True, format='%Y%m%d')
    df = df.sort_values(by='Date').reset_index(drop=True)
    df['Date'] = df['Date'].dt.strftime('%m/%d/%Y')

    data = np.ones((len(df), 1))*1500
    df['winner_elo'] = data
    df['loser_elo'] = data
    
    return df

### Calculating ELO for the first dataset (2010)

In [28]:
df_list = []

df0 = df_preprocessing(files_new[0])
df0 = apply_elo(df0)
df_list.append(df0)

### Calculating ELOs for the remaining years (2011-2021)

In [34]:
for c in range(1, len(files_new)):
    
    df_year_earlier = df_list[c-1]
    df = df_preprocessing(files_new[c])
    df = apply_elo_on_new_season(df, df_year_earlier)
    df = apply_elo(df)
    df_list.append(df)

### Extracting new dataframes as excel tables

In [35]:
for j in range(len(files_new)):
    #doc_name = files_new[j].split('\\')[-1].split('.')[0] + '_with_elo.xlsx'
    doc_name = files_new[j] + '_with_elo.xlsx'
    df_list[j].to_excel(doc_name, index=False)